## 深圳链家二手房

* 深圳链家二手房房源数据分析；

* 关于数据集介绍可以参考[深圳某家网二手房数据（带经纬度） ](https://www.heywhale.com/mw/dataset/60d49c7b056f570017c2009c)


In [1]:
import pandas as pd
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode

## 数据处理

爬虫爬取的数据，通常都会比较“脏”，需要我们先进行部分数据的处理；

In [2]:
# 数据处理
# 读取数据，编码指定为GBK
df = pd.read_csv('/home/mw/input/house4500/二手房数据-sz.csv', encoding='GBK')

# 替换掉数据中的文本，并转为int
df['参考单价'] = df['参考单价'].str.replace('元/平米', '').astype('int')
df['建筑面积'] = df['建筑面积'].str.replace('㎡', '').astype('float')
df['参考总价'] = df['参考总价'].str.replace('万', '').astype('float')
# 提取是否有抵押
df['抵押信息'] = df['抵押信息'].str.extract('(.+?抵押)')
# 提取楼层
df['所在楼层'] = df['所在楼层'].str.extract('(.+?楼层)')
df['房屋朝向'] = df['房屋朝向'].str.extract('(.+?) ')
df.head()

,小区名称,行政区,区域,编号,参考总价,参考单价,房屋户型,所在楼层,建筑面积,户型结构,...,上次交易时间,房屋用途,房屋年限,产权所属,抵押信息,房本备件,房协编码,经度,纬度,城市
0,铂金时代罗湖,罗湖区,罗湖口岸,1.050000e+11,176.0,49600,1室1厅1厨1卫,低楼层,35.33,平层,...,2010/1/5,普通住宅,满五年,非共有,有抵押,已上传房本照片,U216478561162,114.128566,22.541847,深圳
1,颖隆大厦,罗湖区,布心,1.050000e+11,129.0,39500,1室0厅1厨1卫,低楼层,32.48,平层,...,2003/10/27,普通住宅,满五年,非共有,无抵押,已上传房本照片,U218493148084,114.126924,22.586033,深圳
2,颖隆大厦,罗湖区,布心,1.050000e+11,145.0,39500,1室0厅1厨1卫,低楼层,36.70,平层,...,2017/7/14,普通住宅,满两年,非共有,有抵押,已上传房本照片,U195812027604,114.126924,22.586033,深圳
3,梧桐山新居二期,罗湖区,莲塘,1.050000e+11,141.0,57600,1室0厅1厨1卫,低楼层,24.37,平层,...,2003/8/29,普通住宅,满五年,非共有,无抵押,已上传房本照片,1.63E+11,114.181000,22.573630,深圳
4,惠名花园,罗湖区,莲塘,1.050000e+11,130.0,43200,1室1厅1厨1卫,高楼层,29.87,平层,...,2010/4/27,普通住宅,满五年,非共有,有抵押,已上传房本照片,u201569659776,114.190074,22.561214,深圳


## 各行政区单价

房价自然是大家最关心的话题，首先我们来看看深圳各行政区的平均房价是多少；

In [11]:
# 按行政区求平均房价
df_t = df.groupby(['行政区']).agg({'参考单价': 'mean'}).reset_index()

# 将数据转为list
data_pair=[]
for idx, row in df_t.iterrows():
    data_pair.append([row['行政区'], int(row['参考单价'])])

# 新建一个Map
chart = Map(
    init_opts=opts.InitOpts(
        width='1000px',
        height='600px',
        theme='white',
        )
)

# 添加数据
chart.add(
    "",
    data_pair=data_pair,
    maptype="深圳",  # 选择深圳地图
    is_map_symbol_show=False,   # 关闭Map上的红点
    label_opts=opts.LabelOpts(is_show=True)   # 显示标签
)
chart.set_global_opts(
    # 视觉组件设置
    visualmap_opts=opts.VisualMapOpts(
        is_show=False,   # 关闭视觉组件的图例
        type_='color',   # 映射类型选择颜色，可省略
        min_=3e4,    # 设置映射的数据范围
        max_=8e4,
        range_color=[
            "#CCFBFF",
            "#EF96C5"
            ]  # 映射的颜色
    ),
    # 标题设置
    title_opts=opts.TitleOpts(
        title="深圳各行政区二手房房价统计",
        subtitle='单位：元/平方米',
        pos_left="center",
        pos_top='1%',
        title_textstyle_opts=opts.TextStyleOpts(color='#00BFFF', font_size=20)
    ),
    # 提示框配置
    tooltip_opts=opts.TooltipOpts(is_show=True, formatter='{b}:{c}元/平方米'),
    # 关闭图例显示
    legend_opts=opts.LegendOpts(is_show=False),
)

chart.render_notebook()

## 各行政区房价/面积/房源数量概览

通过散点图来展示各行政区放假和户型大小的分布情况；

* X轴对应平均房价，Y轴对应户型的建筑面积，散点大小对应该行政区在售的二手房房源数量；

In [12]:
# 提示框的JS代码
tool_js = """function (param) {return
            '<div style="border-bottom: 1px solid rgba(255,255,255,.3); font-size: 18px;padding-bottom: 7px;margin-bottom: 7px">'
            +param.data[2]+'<br/>'
            + '</div>'
            +'平均面积： '+param.data[1]+' ㎡<br/>'
            +'平均单价： '+param.data[0]+' 元/㎡<br/>'
            +'房源数量： '+param.data[3]+' 套';}"""

# 新建散点图
st = Scatter(
    init_opts=opts.InitOpts(
        theme='white',
        width='1000px',
        height='600px')
)

# 全局配置项
st.set_global_opts(
    # x轴配置
    xaxis_opts=opts.AxisOpts(
        name='单价', 
        type_="value",    # 坐标轴类型为Value
        is_scale=True,   # 为True时，坐标轴起点不固定为0
        # 网格线配置
        splitline_opts=opts.SplitLineOpts(
            is_show=True, 
            linestyle_opts=opts.LineStyleOpts(
                type_='dashed'))
                ),
    # y轴配置，与x轴配置一致
    yaxis_opts=opts.AxisOpts(
        is_scale=True, 
        name='建筑面积', 
        type_="value",
        splitline_opts=opts.SplitLineOpts(
            is_show=True, 
            linestyle_opts=opts.LineStyleOpts(
                type_='dashed'))
                ),
    # 提示框内容配置，使用JS代码
    tooltip_opts=opts.TooltipOpts(formatter=JsCode(tool_js)),
    # 图例设置
    legend_opts=opts.LegendOpts(
        is_show=True,
        pos_right=10,
        pos_top='10%',
        orient='vertical'),
    # 视觉组件配置
    # 包含两块视觉组件映射，图形大小和颜色
    visualmap_opts=[
        opts.VisualMapOpts(
            is_show=True,
            type_='size',
            min_=0,
            max_=10000,
            pos_top='55%',
            pos_right='1%',
            orient='vertical',
            range_text=['房源数量\n', ''],
            range_size=[10, 50],
            textstyle_opts=opts.TextStyleOpts(color='#000')
        ),
        opts.VisualMapOpts(
            is_show=False,
            type_='color',
            dimension=0,
            min_=3e4,
            max_=8e4,
            range_color=[
            "#CCFBFF",
            "#EF96C5"
            ] 
        )],
    # 标题配置
    title_opts=opts.TitleOpts(title="深圳各行政区二手房数据总览",
                              pos_left='center',
                              title_textstyle_opts=opts.TextStyleOpts(color='#1E90FF'))
)

# 数据处理
df_t = df.groupby(['行政区']).agg(
    {'参考单价': 'mean', '建筑面积': 'mean', '编号': 'count'}).reset_index()

# 转为list
data_x, data_y = [], []
for idx, row in df_t.iterrows():
    data_x.append(int(row['参考单价']))
    data_y.append([int(row['建筑面积']), row['行政区'], row['编号']])

# 添加数据
st.add_xaxis(data_x)
st.add_yaxis(
    '',
    data_y,
    # 标签设置，显示行政区名字
    label_opts=opts.LabelOpts(
        is_show=True,
        color='black',
        formatter=JsCode('function (p) {return p.data[2];}')),
    # 图形样式配置
    itemstyle_opts={'opacity': 0.9,  # 透明度
                    'borderColor': '#111111',   # 边框颜色
                    'boderWidth': 3}
)


st.render_notebook()


## 各区域房源数量

看完了行政区维度，再来看看各个区域的房源分布，看看在深圳，那些地块的房产分布最多？

* *可以通过点击下钻到小区维度*


In [5]:
# 分组计数
df_t = df.groupby(['行政区', '区域', '小区名称'])['编号'].count().reset_index()

# TreeMap数据处理
data = []
area, county = [], []
for idx, row in df_t.iterrows():
    if row['行政区'] in county:
        if row['区域'] in area:
            data[-1]['children'][-1]['children'].append(dict(name=row['小区名称'], value=row['编号']))
        else:
            data[-1]['children'].append(dict(name=row['区域'], children=[dict(name=row['小区名称'], value=row['编号'])]))
    else:
        data.append(dict(name=row['行政区'], children=[dict(name=row['区域'], children=[dict(name=row['小区名称'], value=row['编号'])])]))
    area.append(row['区域'])
    county.append(row['行政区'])

# 新建一个TreeMap
tree = TreeMap(    
    init_opts=opts.InitOpts(
        theme='chalk',
        width='1000px',
        height='600px'
        ))

# 添加数据
tree.add(
    "房源分布", 
    data,
    leaf_depth=2, # 显示两层 即最上层包含行政区和热门区域
    label_opts=opts.LabelOpts(position="inside", formatter='{b}: {c}套'),  # 标签样式设置
    levels=[
            opts.TreeMapLevelsOpts(  # 第一层样式设置，即行政区级别
                treemap_itemstyle_opts=opts.TreeMapItemStyleOpts(
                    border_color="#555", border_width=4, gap_width=4
                )
            ),
            opts.TreeMapLevelsOpts(  # 第二层样式设置，即行政区内的区域
                color_saturation=[0.3, 0.6],
                treemap_itemstyle_opts=opts.TreeMapItemStyleOpts(
                    border_color_saturation=0.7, gap_width=2, border_width=2
                ),
            ),
            opts.TreeMapLevelsOpts(  # 第三层样式设置，即小区
                color_saturation=[0.3, 0.5],
                treemap_itemstyle_opts=opts.TreeMapItemStyleOpts(
                    border_color_saturation=0.6, gap_width=1
                ),
            ),
        ],
    )
tree.set_global_opts(
    # 标题设置
    title_opts=opts.TitleOpts(
        title="深圳各区块二手房房源分布", 
        pos_left='center', 
        pos_top='2%', 
        title_textstyle_opts=opts.TextStyleOpts(color='#00BFFF', font_size=20)
        ),
    # 关闭图例显示
    legend_opts=opts.LegendOpts(is_show=False)
    )

tree.render_notebook()

## 各小区房价对比

通过Map3D图表，可以清晰的看到，在售房源中，大部分都集中于南山和福田，而且平均房价也高出其他区域不少；

In [6]:
# 计算小区单机和经纬度，经纬度取该小区内多套房源的平均值
df_t = df.groupby(['小区名称']).agg(
    {'参考单价': 'mean', '经度': 'mean', '纬度': 'mean'}).reset_index()

data_pair=[]
for idx, row in df_t.iterrows():
    # map3d需要的数据格式 [[名称，经度，维度，数值]，]
    data_pair.append([row['小区名称'], [row['经度'], row['纬度'], int(row['参考单价'])]])

# 新建一个Map3D
chart = Map3D(
    init_opts=opts.InitOpts(
        width='1000px',
        height='600px',
        theme='chalk',
        )
)
# 引用添加的地图
chart.add_schema(
    maptype="深圳",
    ground_color='#999',  # 地面颜色
    shading="lambert",  # 着色效果
    box_width=130,   # 整个组件的宽
    box_height=30,   # 整个组件的高
    emphasis_label_opts=opts.LabelOpts(is_show=True),  # 选中区域后的效果
    itemstyle_opts=opts.ItemStyleOpts(  # 图形样式
            border_width=0.2,   
            border_color="rgb(0,0,0)",
    ),
    # 光源配置
    light_opts=opts.Map3DLightOpts(
        main_shadow_quality='high',  # 光影质量
        is_main_shadow=True,   
        main_intensity=1,  # 环境光强度
        main_alpha=30,  # 光源角度
        ambient_cubemap_texture='https://echarts.apache.org/examples/data-gl/asset/canyon.hdr',  # HDR效果
        ambient_cubemap_diffuse_intensity=0.5,   # 漫反射强度
        ambient_cubemap_specular_intensity=0.5   # 高光反射强度
    ),
    # 后处理特效
    post_effect_opts=opts.Map3DPostEffectOpts(
        is_enable=True,   # 是否开启后处理特效
        is_ssao_enable=True,     # 是否开启环境光遮蔽效果
        ssao_radius=1,  # 环境光遮蔽的采样半径
        ssao_intensity=1   # 环境光遮蔽强度
    )
)
chart.add(
    "",
    data_pair=data_pair,
    type_="bar3D",   # 类型使用3D柱状图
    bar_size=0.3,   # bar的大小，越小越细
    min_height=1,   # 最低高度
    shading="lambert",   # 着色效果
    label_opts=opts.LabelOpts(
            is_show=False,
            formatter=JsCode(
                "function(data){return data.name + ': ' + data.value[2]+'元/平方米';}"),
    )   # 标签配置
)
chart.set_global_opts(
    # Bar对应的视觉组件
    visualmap_opts=opts.VisualMapOpts(
        is_show=False,
        type_='color',
        dimension=2,   # 指定维度
        min_=3e4,
        max_=1e5,
        range_color=[
            '#313695',
            '#4575b4',
            '#74add1',
            '#abd9e9',
            '#e0f3f8',
            '#ffffbf',
            '#fee090',
            '#fdae61',
            '#f46d43',
            '#d73027',
            '#a50026']
    ),
    # 标题配置
    title_opts=opts.TitleOpts(
        title="深圳各小区二手房房价统计",
        pos_left="center",
        pos_top='1%',
        title_textstyle_opts=opts.TextStyleOpts(color='#00BFFF', font_size=20)
    ),
    # 提示框配置
    tooltip_opts=opts.TooltipOpts(is_show=False),
    # 图例配置
    legend_opts=opts.LegendOpts(is_show=False),
)

chart.render_notebook()

## 房源属性

对比不同属性下的房源价格差别

In [7]:
# 会多次复用的代码，直接使用函数
# 返回一个Bar
# desc 用于标题和数据筛选
# title_pos  指定标题显示位置
def bar_chart(desc, title_pos):
    df_t = df.groupby([desc]).agg({'参考单价': 'mean', '编号':'count'}).reset_index()
    # 保证该属性下至少100套房源才参与对比
    df_t = df_t[df_t['编号']>100]  
    df_t['参考单价'] = df_t['参考单价'].astype('int')
    df_t = df_t.sort_values('参考单价')

    # 新建一个Bar
    chart = Bar()
    chart.add_xaxis(
        df_t[desc].tolist()
    )
    chart.add_yaxis(
        '',
        df_t['参考单价'].tolist()
    )
    
    # Bar的全局配置项
    chart.set_global_opts(
        xaxis_opts=opts.AxisOpts(
            name='属性',
            is_scale=True,
            # 网格线配置
            splitline_opts=opts.SplitLineOpts(
                is_show=True,
                linestyle_opts=opts.LineStyleOpts(
                    type_='dashed'))
        ),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            name='单价',
            type_="value",
            # 网格线配置
            splitline_opts=opts.SplitLineOpts(
                is_show=True,
                linestyle_opts=opts.LineStyleOpts(
                    type_='dashed'))
        ),
        # 标题配置
        title_opts=opts.TitleOpts(
            title=desc,
            pos_left=title_pos[0],
            pos_top=title_pos[1],
            title_textstyle_opts=opts.TextStyleOpts(color='#00BFFF', font_size=20)
        ),
    )
    return chart


In [8]:
# 新建组合图表Grid
grid = Grid(
    init_opts=opts.InitOpts(
        theme='white',
        width='1000px',
        height='1000px')
)
# 依次添加不同属性下价格对比Bar
grid.add(
    bar_chart('装修情况', ['5%', '3%']),
    is_control_axis_index=False,
    grid_opts=opts.GridOpts(
        pos_top='10%',   # 指定Grid中子图的位置
        pos_bottom='70%',
        pos_left='10%',
        pos_right='60%'
    )
)
grid.add(
    bar_chart('所在楼层', ['55%', '3%']),
    is_control_axis_index=False,
    grid_opts=opts.GridOpts(
        pos_top='10%',
        pos_bottom='70%',
        pos_left='60%',
        pos_right='10%'
    )
)
grid.add(
    bar_chart('房屋朝向', ['5%', '33%']),
    is_control_axis_index=False,
    grid_opts=opts.GridOpts(
        pos_top='40%',
        pos_bottom='40%',
        pos_left='10%',
        pos_right='60%'
    )
)
grid.add(
    bar_chart('房屋年限', ['55%', '33%']),
    is_control_axis_index=False,
    grid_opts=opts.GridOpts(
        pos_top='40%',
        pos_bottom='40%',
        pos_left='60%',
        pos_right='10%'
    )
)
grid.add(
    bar_chart('户型结构', ['5%', '63%']),
    is_control_axis_index=False,
    grid_opts=opts.GridOpts(
        pos_top='70%',
        pos_bottom='10%',
        pos_left='10%',
        pos_right='60%'
    )
)
grid.add(
    bar_chart('建筑结构', ['55%', '63%']),
    is_control_axis_index=False,
    grid_opts=opts.GridOpts(
        pos_top='70%',
        pos_bottom='10%',
        pos_left='60%',
        pos_right='10%'
    )
)
grid.render_notebook()


## Task 今日份小作业

* 使用Grid组合图表，对比不同房屋属性下房屋总价的差距（包含装修情况，房屋朝向，户型结构，建筑结构，房屋年限，所在楼层，抵押信息，配备电梯总共8个属性，组合图表使用3X3布局）；

![Image Name](https://cdn.kesci.com/upload/image/refssuh79b.png?imageView2/0/w/960/h/960)


* 使用Map3d对比各小区平均房源**总价**；

![Image Name](https://cdn.kesci.com/upload/image/refsthxpb5.png?imageView2/0/w/960/h/960)
